In [1]:
import os
import numpy as np
import datetime

import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

from torch_mlp import MLP
from torch_segment_dataset import SegmentDataset

def metricDist(y_true, y_pred):
    row = (y_pred[:,:,:,0] - y_true[:,:,:,0])**2
    col = (y_pred[:,:,:,1] - y_true[:,:,:,1])**2
    return torch.mean(row + col) ** 0.5

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(f"use_cuda: {use_cuda}, device: {device}")

data_dir = "data/geolife/Data/"

user_list = ["068", "003", "004"]
min_length = 6
time_delta = 60
length = min_length * time_delta

y_timestep = min_length * 2
x_attribute = 20
label_attribute = 2
sample_s = 10
replace = False

batch_size = 2

# Dataset
training_data    = SegmentDataset(data_dir, user_list, device, time_delta, y_timestep, length, label_attribute, sample_s, replace)
train_dataloader = DataLoader(training_data, batch_size, shuffle=False)

# Model 훈련 진행
model = MLP(input_shape=[(length-y_timestep), x_attribute], y_timestep = y_timestep, label_attribute=label_attribute)

model = model.to(torch.double)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(10):
    model.train()
    for train_idx, train_data in enumerate(train_dataloader, 0):
        task_X, task_y = train_data
        optimizer.zero_grad()
        output = model(task_X)
        loss = criterion(output, task_y)
        loss.backward()
        optimizer.step()

# 예측
with torch.no_grad():
    model.eval()
    predicted = model(task_X)
    loss = metricDist(task_y, predicted)
    print("target latitude, longitude:", task_y)
    print("Predicted latitude, longitude:", predicted)
    print(f'loss: {loss}')


use_cuda: True, device: cuda:0
target latitude, longitude: tensor([[[[  57.,  101.],
          [  57.,  101.],
          [  57.,  101.],
          [  57.,  101.],
          [  57.,  101.],
          [  57.,  101.],
          [  57.,  101.],
          [  57.,  101.],
          [  57.,  101.],
          [  57.,  101.],
          [  57.,  101.],
          [  58.,  101.]],

         [[  62.,   81.],
          [  62.,   81.],
          [  62.,   81.],
          [  62.,   81.],
          [  62.,   81.],
          [  63.,   81.],
          [  63.,   81.],
          [  63.,   81.],
          [  63.,   81.],
          [  63.,   81.],
          [  64.,   81.],
          [  64.,   81.]],

         [[ 883., 2784.],
          [ 883., 2784.],
          [ 883., 2784.],
          [ 883., 2784.],
          [ 883., 2784.],
          [ 883., 2784.],
          [ 882., 2784.],
          [ 882., 2784.],
          [ 882., 2784.],
          [ 882., 2784.],
          [ 882., 2784.],
          [ 882., 2784.]],


In [ ]:
import pandas as pd

writer_dir_name = 'data/geolife/runs'
log_dir_name = '[grid_origin_1000]_20240224-173505'
log_folder = writer_dir_name + '/' + log_dir_name
# best_model_path = log_folder + '/best_model.pth'
best_model_path = log_folder + '/best_train_model.pth'

config_df = pd.read_csv(log_folder + '/configuration.csv')
config_df.head()

In [ ]:
# import pandas as pd

# writer_dir_name = 'data/geolife/runs'
# log_dir_name = '[grid_origin_1000]_20240222-162606'
# log_folder = writer_dir_name + '/' + log_dir_name
# # best_model_path = log_folder + '/best_model.pth'
# best_model_path = log_folder + '/best_train_model.pth'

# config_df = pd.read_csv(log_folder + '/configuration.csv')

data_dir = "data/geolife/Data/"
label_attribute = 2

sample_s = config_df['sample_s'][0]
sample_q = config_df['sample_q'][0]

args_epoch = config_df['epoch'][0]
args_patience = config_df['patience'][0]

gap_min = 12 # 1 min
gap = gap_min

day = config_df['day'][0]
week = config_df['week'][0]
y_timestep = config_df['y_timestep'][0]   #12 # must be less than length, 12 = 1 hour, 12 * 24 = 288 -> 1day
length = config_df['length'][0] # -> 4 weeks

train_list = config_df['train_list'][0]
validation_list = config_df['val_list'][0]
test_list = config_df['test_list'][0]

train_size = 0.4
validation_size = 0.1
batch_size = 1 # each user

## Test Phase
is_train = False

print(f"y_timestep: {y_timestep}, length: {length}")

In [ ]:
# TensorBoard 설정
import ast
from torch_time_het import TimeHetNet

import torch
from torch.utils.data import DataLoader
from torch_geolife_dateset import GeoLifeDataSet
from data.geolife.convert_minmax_location import LocationPreprocessor
import random

data_dir = "data/geolife/Data/"


train_list = config_df['train_list'][0]
validation_list = config_df['val_list'][0]
test_list = config_df['val_list'][0]#, '078']

test_list = ['068', '038']#, '039']#, '067']

print(f"train_list:      {train_list}")
print(f"validation_list: {validation_list}")
print(f"test_list:       {test_list}")

# Multi user
training_data   = GeoLifeDataSet(data_dir, train_list, sample_s, sample_q, length, y_timestep, gap, label_attribute)
validation_data = GeoLifeDataSet(data_dir, validation_list, sample_s, sample_q, length, y_timestep, gap, label_attribute)
test_data       = GeoLifeDataSet(data_dir, test_list, sample_s, sample_q, length, y_timestep, gap, label_attribute)

# single user
# training_data   = GeoLifeDataSet(data_dir, train_list, sample_s, sample_q, length, y_timestep, gap, label_attribute, mode='train')
# validation_data = GeoLifeDataSet(data_dir, validation_list, sample_s, sample_q, length, y_timestep, gap, label_attribute, mode='valid')
# test_data       = GeoLifeDataSet(data_dir, test_list, sample_s, sample_q, length, y_timestep, gap, label_attribute, mode='valid')


train_dataloader      = DataLoader(training_data, batch_size, shuffle=False)
validation_dataloader = DataLoader(validation_data, batch_size, shuffle=False)
test_dataloader       = DataLoader(test_data, batch_size, shuffle=False)

best_model = TimeHetNet(dims_inf = ast.literal_eval("[32,32,32]"),
                        dims_pred = ast.literal_eval("[32,32,32]"), 
                        activation="relu", 
                        time=100,
                        batchnorm=False, 
                        block = str("gru,conv,conv,gru").split(","),
                        output_shape=[y_timestep, 2],
                        length=length)
best_model.load_state_dict(torch.load(best_model_path))


In [ ]:
# https://jimmy-ai.tistory.com/30
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

class_probs = []
class_label = []
que_x_list = []

with torch.no_grad():
    for idx, test_data in enumerate(test_dataloader, 0):
        test_X, test_y = test_data
        if len(test_X) < 2:
            continue
        class_probs = []
        class_label = []
        que_x_list = []

        que_x, _, _ = test_X
        output = best_model(test_X)
        class_probs.append(output.type(torch.int64))
        class_label.append(test_y)
        # print(que_x.shape)
        que_x_list.append(que_x[:, :, -y_timestep:, 0])
        # print(que_x[:, :, -y_timestep:, :])
        
        test_probs = torch.cat(class_probs)
        test_label = torch.cat(class_label)
        test_que_x = torch.cat(que_x_list)

        # print(test_probs.shape)
        # print(test_label.shape)
        # print(test_que_x.shape)

        for user_idx in range(test_label.shape[0]):
            for test_idx in range(test_label.shape[1]):
                df = pd.DataFrame(data=test_que_x[user_idx][0], columns=['Time'])
                df_norm = MinMaxScaler().fit_transform(df)

                plt.figure(figsize=(10, 5))
                plt.subplot(2, 2, 1)
                plt.title('Row graph')
                # plt.scatter(df_norm, test_label[user_idx][test_idx][:, 0] , color='r', alpha=0.5)
                # plt.scatter(df_norm, test_probs[user_idx][test_idx][:, 0], color='g', alpha=0.3)
                plt.plot(df_norm, test_label[user_idx][test_idx][:, 0] , color='r', alpha=0.5)
                plt.plot(df_norm, test_probs[user_idx][test_idx][:, 0], color='g', alpha=0.3)
                plt.xlabel('time')
                plt.ylabel('grid_row')
                
                # fig_col = plt.figure(figsize=(5, 3))
                plt.subplot(2, 2, 2)
                plt.title('Column graph')
                # plt.scatter(df_norm, test_label[user_idx][test_idx][:, 1] , color='r', alpha=0.5)
                # plt.scatter(df_norm, test_probs[user_idx][test_idx][:, 1], color='g', alpha=0.3)
                plt.plot(df_norm, test_label[user_idx][test_idx][:, 1] , color='r', alpha=0.5)
                plt.plot(df_norm, test_probs[user_idx][test_idx][:, 1], color='g', alpha=0.3)
                plt.xlabel('time')
                plt.ylabel('grid_col')
                # plt.show() 


In [ ]:
# https://jimmy-ai.tistory.com/30
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

for idx in range(test_label.shape[0]):
    df = pd.DataFrame(data=test_que_x[idx][0], columns=['Time'])
    df_norm = MinMaxScaler().fit_transform(df)

    fig_row = plt.figure(figsize=(9, 6))
    ax_row = fig_row.add_subplot(111)
    ax_row.plot(df_norm, test_label[idx][0][:, 0] , color='r', alpha=0.5)
    ax_row.plot(df_norm, test_probs[idx][0][:, 0], color='g', alpha=0.3)
    ax_row.set_xlabel('time')
    ax_row.set_ylabel('grid_row')
    
    ax_col = fig_row.add_subplot(222)
    ax_col.plot(df_norm, test_label[idx][0][:, 1] , color='r', alpha=0.5)
    ax_col.plot(df_norm, test_probs[idx][0][:, 1], color='g', alpha=0.3)
    ax_col.set_xlabel('time')
    ax_col.set_ylabel('grid_col')


In [ ]:
# https://jimmy-ai.tistory.com/30
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# fig = plt.figure(figsize=(9, 6))
# ax = fig.add_subplot(111, projection='3d')

# ax.scatter(x, y, z, color = 'r', alpha = 0.5)
# ax.scatter(x, z, y, color = 'g', alpha = 0.5) # y와 z축 swap

for idx in range(test_label.shape[0]):
    fig = plt.figure(figsize=(9, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=10, azim=-50)
    df = pd.DataFrame(data=test_que_x[idx][0], columns=['Time'])
    df_norm = MinMaxScaler().fit_transform(df)
    ax.scatter(test_label[idx][0][:, 0], test_label[idx][0][:, 1], df_norm,
               color='r', alpha=0.5)
    ax.scatter(test_probs[idx][0][:, 0], test_probs[idx][0][:, 1], df_norm, 
               color='g', alpha=0.5)
    print(df_norm)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('time')

In [ ]:
# 헬퍼 함수
def add_pr_curve_tensorboard(class_index, test_probs, test_label, global_step=0):
    '''
    0부터 9까지의 "class_index"를 가져온 후 해당 정밀도-재현율(precision-recall)
    곡선을 그립니다
    '''
    tensorboard_truth = test_label == class_index
    tensorboard_probs = test_probs[:, class_index]

    writer.add_pr_curve(classes[class_index],
                        tensorboard_truth,
                        tensorboard_probs,
                        global_step=global_step)
    writer.close()

# 모든 정밀도-재현율(precision-recall; pr) 곡선을 그립니다
for i in range(len(classes)):
    add_pr_curve_tensorboard(i, test_probs, test_preds)


In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.functional as F

from torch import optim

import pandas as pd

class My_Linear(nn.Module):
    def __init__(self, in_features, out_features, bias=True, device=None, name=None):
        super(My_Linear, self).__init__()
        self.Linear = nn.Linear(in_features, out_features, bias, device)
        self.name = name
    
    def forward(self, x):
        # print(f"My_Linear - X.shape: {x.shape}, name: {self.name}")
        x = self.Linear(x)
        return x

In [ ]:
df = pd.read_csv('data/geolife/Data/000/csv/000.csv')
df.head(1)
df_test = df.iloc[:, :-2].copy()
df_test.head(1)

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [ ]:
X = ['latitude', 'longitude', 'altitude', 'days']
df_x = df_test[X].head(10).copy()
df_x = df_to_tensor(df_x)
print(df_x)
df_x = df_x.unsqueeze(axis=-1)
print(df_x)
df_x = df_x.sum(axis=2)
print(df_x)

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df_test, train_size=0.7, random_state=123)

X = ['latitude', 'longitude', 'altitude', 'days']
y = ['latitude', 'longitude']

row = 10
train_set = train_set.iloc[:row, :].copy()

train_X = df_to_tensor(train_set[X])
train_y = df_to_tensor(train_set[y])

model = My_Linear(4, 2)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
out_list = []
loss_train = 0.0
epochs = 10
for epoch in range(epochs):
    for input in train_X:
        outputs = model(input)
        print(outputs)
        loss = loss_fn(outputs, train_y)
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
    
    print(f"epoch:{epoch}, loss:{loss_train}")

In [ ]:
train_y